# Intro

This notebook applies the intensity gradient based segmentation process to create masks of individual embryos

The outputs of this notebook and [1b_gradient_segmentation](1b_gradient_segmentation.ipynb) are combined (in downstream processes) to generate the final segmentation mask

# Imports

In [ ]:
import os
import shutil
import numpy as np
import cv2
from joblib import delayed, Parallel
from tqdm import tqdm

from fam13a import utils, image

# Constants

In [ ]:
PROJ_ROOT = utils.here()
DATA_ROOT = os.path.join(PROJ_ROOT, 'data', 'interim', 'xenopus')
OUTPUT_ROOT = os.path.join(PROJ_ROOT, 'data', 'processed', 'xenopus', 'segmented', 'gradient')

# set level of parallelisation
NCPUS = 8

# define the cropping to apply to all frames in a video
# use None to signify no cropping as slice(0, 0) will pick out an empty array
# we need to crop the Xenopus videos because there is a timer added to the top-right corner all videos
# which heavily skews any adaptive calculation in the segmentation process
CROPS = (slice(50, None), slice(None, None)) 

file_ids = [_f.split('.mp4')[0] for _f in os.listdir(DATA_ROOT) if _f.endswith('mp4')]
print(file_ids)

# Setup

In [ ]:
# define a simple wrapper function for the segmentation to make it easy to parallelise
def process(frame, label, output, zfill_val):
    # apply the segmentation algorithm to a single frame
    mask = image.segment.gradient(frame)
    # pad the output name of the file with 0's so when they are sorted alphabetically
    # they are in the correct order
    name = f'{label}'.zfill(zfill_val)
    np.save(os.path.join(output, name), mask)

# Processing

Each video takes ~10min to process on 8 CPUs

In [ ]:
# loop over each input video
for file_count, file_id in enumerate(file_ids):
    print(f'Started processing: {file_id} ({file_count+1} of {len(file_ids)})')
    # ensure output directory exists and it is empty
    output = os.path.join(OUTPUT_ROOT, file_id)
    try:
        shutil.rmtree(output)
    except FileNotFoundError:
        pass
    os.makedirs(output)
    
    # load video frames
    frames = utils.frames_from_video(os.path.join(DATA_ROOT, f'{file_id}.mp4'))
    # calculate the zfill value to use on the output file names so they are 
    # sorted alphanumerically by name
    zfill_val = len(str(frames.shape[0]))
    
    # convert colour images to grayscale
    frames = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in frames]
    # estimate and remove the background from each frame using a patched approach
    frames = image.background.remove_patched(frames, image.consts.XENOPUS_BCKGR_PATCH_SHAPE,
                                             invert=True, ncpus=NCPUS)

    # crop all frames in the video to remove the timer in the top-right corner
    frames = np.stack(frames, axis=0)[(..., *CROPS)]

    with Parallel(n_jobs=NCPUS, verbose=1) as par:
        par(delayed(process)(frame, label, output, zfill_val) for label, frame in enumerate(frames))